<a href="https://colab.research.google.com/github/S-Abhishekk/Celebal-Assignments-/blob/main/Assignment7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install streamlit pyngrok scikit-learn pandas matplotlib joblib -q


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 62.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 62.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 5.1 MB/s eta 0:00:00


In [4]:
from sklearn.datasets import load_iris
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import pandas as pd
import joblib

# Load dataset (no CSV needed)
iris = load_iris()
df = pd.DataFrame(iris.data, columns=iris.feature_names)
df['Species'] = pd.Categorical.from_codes(iris.target, iris.target_names)

# Train-test split
X = df.drop(columns=["Species"])
y = df["Species"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Train model
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

# Save model
joblib.dump(clf, "model.pkl")
print("Model saved as model.pkl")


Model saved as model.pkl


In [7]:
%%writefile app.py
import streamlit as st
import joblib
import pandas as pd
import matplotlib.pyplot as plt

# Load the trained model
@st.cache_resource
def load_model():
    return joblib.load("model.pkl")

model = load_model()

st.title("🌸 Iris Species Predictor (Colab Demo)")
st.markdown("Use the sliders in the sidebar to input flower measurements and predict its species.")

# Sidebar for user inputs
st.sidebar.header("Input Features")
def get_user_input():
    return {
        "sepal length (cm)": st.sidebar.slider("Sepal Length", 4.0, 8.0, 5.0),
        "sepal width (cm)": st.sidebar.slider("Sepal Width", 2.0, 4.5, 3.0),
        "petal length (cm)": st.sidebar.slider("Petal Length", 1.0, 7.0, 4.0),
        "petal width (cm)": st.sidebar.slider("Petal Width", 0.1, 2.5, 1.0),
    }

input_data = get_user_input()
input_df = pd.DataFrame([input_data])

st.subheader("Your Input Features")
st.write(input_df)

# Prediction button
if st.button("Predict"):
    pred = model.predict(input_df)[0]
    proba = model.predict_proba(input_df)[0]

    st.success(f"Prediction: **{pred}**")

    # Show probability as a bar chart
    st.subheader("Prediction Confidence")
    fig, ax = plt.subplots()
    ax.bar(model.classes_, proba, color="skyblue", edgecolor="black")
    ax.set_ylabel("Probability")
    ax.set_xlabel("Class")
    st.pyplot(fig)


Overwriting app.py


In [12]:
!ngrok config add-authtoken 309TCHK3gmzUlSpuGjeNVEndWnY_6uirYdACFGGidE4P99AQ


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [13]:
from pyngrok import ngrok
public_url = ngrok.connect(8501)
print(f"Access your app here: {public_url}")
!streamlit run app.py --server.port 8501 &> /dev/null &


Access your app here: NgrokTunnel: "https://623ed6674d65.ngrok-free.app" -> "http://localhost:8501"
